In [1]:
import cloudknot as ck
ck.set_region('us-west-2')

In [5]:
def afq_cam_can(subject):
    import fury
    import xvfbwrapper
    import AFQ.data as afqd
    import AFQ.api as api
    import AFQ.mask as afm
    import os.path as op
    import s3fs

    local_bids_folder = "./cam_can"
    bucket = "cam-can-mri"
    scope = 'qsiprep'

    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=[subject],
        anon=False,
        random_seed=42)
    study.download(
        local_bids_folder,
        include_derivs="qsiprep-ck")
    
    tracking_params = {
        "n_seeds": 1, # also try 2
        "directions": "prob",
        "odf_model": "CSD", # also try dki
        "seed_mask": afm.MaskFile("WM_probseg", scope=scope), 
        "seed_treshold": 0.1,
        "stop_mask": afm.ScalarMask("dki_fa"),
        "stop_threshold": 0.2,
    }
    myafq = api.AFQ(
        local_bids_folder,
        dmriprep=scope,
        brain_mask=afm.MaskFile("brain_mask", scope=scope),
        scalars=['dki_md', 'dki_fa'],
        tracking_params=tracking_params,
        virtual_frame_buffer=True)
    myafq.export_all()

    s3fs.S3FileSystem().put(
        op.join(local_bids_folder, "afq"),
        op.join(bucket, s3_prefix, "afq"),
        recursive=True)
    

In [3]:
#afq_cam_can(1)

In [4]:
afq_knot = ck.Knot(name='afq_cam_can-64gb-20819-0',
                  func=afq_cam_can,
                  base_image='libglxvfb:1',
                  image_github_installs="https://github.com/36000/pyAFQ.git@mask_files",
                  pars_policies=('AmazonS3FullAccess',),
                  resource_type="SPOT",
                  bid_percentage=100,
                  memory=64000)

BuildError: pull access denied for libglxvfb, repository does not exist or may require 'docker login': denied: requested access to the resource is denied

In [ ]:
inputs = [1, 2, 3]

In [ ]:
ft = afq_knot.map(inputs)

In [ ]:
afq_knot.view_jobs()